In [ ]:
import pprint
import numpy as np
import pandas as pd

pp = pprint.PrettyPrinter()

crimes = pd.DataFrame.from_csv('crimes_raw.csv', index_col=None)

In [ ]:
# Filter crimes with missing neighborhood information
crimes = crimes[crimes.neighborhood != '-']

In [ ]:
# Define districts from neighborhood data
# District definitions sourced from:
# http://www.cincinnati-oh.gov/police/districts/ 
# https://en.wikipedia.org/wiki/List_of_Cincinnati_neighborhoods
district_1 = [
 'C. B. D. / RIVERFRONT',
 'MOUNT  ADAMS',
 'OVER-THE-RHINE',
 'PENDLETON',
 'QUEENSGATE',
 'WEST  END'
]
district_2 = [
 'CALIFORNIA',
 'COLUMBIA / TUSCULUM',
 'EAST  END',
 'EAST WALNUT HILLS',
 'EVANSTON',
 'HYDE PARK',
 'KENNEDY  HEIGHTS',
 'LINWOOD',
 'MADISONVILLE',
 'MT.  LOOKOUT',
 'MT.  WASHINGTON',
 "O'BRYONVILLE",
 'OAKLEY',
 'PLEASANT RIDGE'
]
district_3 = [
 'EAST PRICE HILL',
 'EAST  WESTWOOD',
 'ENGLISH  WOODS',
 'LOWER PRICE  HILL',
 'MILLVALE',
 'NORTH FAIRMOUNT',
 'RIVERSIDE',
 'SAYLER  PARK',
 'SEDAMSVILLE',
 'S.. CUMMINSVILLE',
 'SOUTH  FAIRMOUNT',
 'FAY APARTMENTS',
 'WEST PRICE HILL',
 'WESTWOOD'
]
district_4 = [
 'AVONDALE',
 'BONDHILL',
 'CARTHAGE',
 'CORRYVILLE',
 'HARTWELL',
 'MOUNT  AUBURN',
 'NORTH AVONDALE',
 'PADDOCK  HILLS',
 'ROSELAWN',
 'WALNUT HILLS'
]
district_5 = [
 'CAMP  WASHINGTON',
 'CLIFTON',
 'COLLEGE  HILL',
 'CLIFTON/UNIVERSITY HEIGHTS',
 'FAIRVIEW',
 'MOUNT AIRY',
 'NORTHSIDE',
 'Spring Grove Village',
 'Winton Hills'
]

# Create master list for later reference
all_districts = sorted(district_1 + district_2 + district_3 + district_4 + district_5)


# Define placeholder for district mappings
district_mappings = pd.DataFrame(columns=['neighborhood', 'district_clean'])

# Set up list for loop
districts = [district_1, district_2, district_3, district_4, district_5]

# Create dataframe, assign district number, and append to the placeholder
for idx, district in enumerate(districts):
    district = pd.DataFrame(district, columns = ['neighborhood'])
    district['district_clean'] = idx + 1
    district_mappings = district_mappings.append(district, ignore_index=True)

In [ ]:
# # Handy generator for building a scaffold for name mappings
# mappings = []
# for district in all_districts:
#     mappings.append([district, ''])
# mappings

# Define mappings between names in data and City of Cincinnati/Census neighbourhoods
# Neighbourhood names sourced from:
# http://www.cincinnati-oh.gov/planning/reports-data/census-demographics/
neighborhood_name_mappings = pd.DataFrame([
    ['AVONDALE', 'Avondale'],
    ['BONDHILL', 'Bond Hill'],
    ['C. B. D. / RIVERFRONT', 'Downtown'],
    ['CALIFORNIA', 'California'],
    ['CAMP  WASHINGTON', 'Camp Washington'],
    ['CARTHAGE', 'Carthage'],
    ['CLIFTON', 'Clifton'],
    ['CLIFTON/UNIVERSITY HEIGHTS', 'CUF'],
    ['COLLEGE  HILL', 'College Hill'],
    ['COLUMBIA / TUSCULUM', 'Columbia Tusculum'],
    ['CORRYVILLE', 'Corryville'],
    ['EAST  END', 'East End'],
    ['EAST  WESTWOOD', 'East Westwood'],
    ['EAST PRICE HILL', 'East Price Hill'],
    ['EAST WALNUT HILLS', 'East Walnut Hills'],
    ['ENGLISH  WOODS', 'English Woods'],
    ['EVANSTON', 'Evanston'],
    ['FAIRVIEW', 'CUF'],
    ['FAY APARTMENTS', 'Villages at Roll Hill'],
    ['HARTWELL', 'Hartwell'],
    ['HYDE PARK', 'Hyde Park'],
    ['KENNEDY  HEIGHTS', 'Kennedy Heights'],
    ['LINWOOD', 'Linwood'],
    ['LOWER PRICE  HILL', 'Lower Price Hill'],
    ['MADISONVILLE', 'Madisonville'],
    ['MILLVALE', 'Millvale'],
    ['MOUNT  ADAMS', 'Mt. Adams'],
    ['MOUNT  AUBURN', 'Mt. Auburn'],
    ['MOUNT AIRY', 'Mt. Airy'],
    ['MT.  LOOKOUT', 'Mt. Lookout'],
    ['MT.  WASHINGTON', 'Mt. Washington'],
    ['NORTH AVONDALE', 'North Avondale - Paddock Hills'],
    ['NORTH FAIRMOUNT', 'North Fairmount'],
    ['NORTHSIDE', 'Northside'],
    ["O'BRYONVILLE", 'Evanston'],
    ['OAKLEY', 'Oakley'],
    ['OVER-THE-RHINE', 'Over-the-Rhine'],
    ['PADDOCK  HILLS', 'North Avondale - Paddock Hills'],
    ['PENDLETON', 'Pendleton'],
    ['PLEASANT RIDGE', 'Pleasant Ridge'],
    ['QUEENSGATE', 'Queensgate'],
    ['RIVERSIDE', 'Riverside'],
    ['ROSELAWN', 'Roselawn'],
    ['S.. CUMMINSVILLE', 'South Cumminsville'],
    ['SAYLER  PARK', 'Sayler Park'],
    ['SEDAMSVILLE', 'Sedamsville'],
    ['SOUTH  FAIRMOUNT', 'South Fairmount'],
    ['Spring Grove Village', 'Spring Grove Village'],
    ['WALNUT HILLS', 'Walnut Hills'],
    ['WEST  END', 'West End'],
    ['WEST PRICE HILL', 'West Price Hill'],
    ['WESTWOOD', 'Westwood'],
    ['Winton Hills', 'Winton Hills']
], columns=['neighborhood', 'neighborhood_clean'])

In [ ]:
# Join the two mappings (neighborhoods and districts, neighborhoods and cleaned-up)
neighborhood_name_district_mappings = neighborhood_name_mappings.merge(district_mappings, on='neighborhood')

# Join the mappings back onto the original data
crimes_merged = crimes.merge(neighborhood_name_district_mappings, on='neighborhood')

# Write to file
crimes_merged.to_csv('crimes_clean.csv', index=False)

In [ ]:
# Get counts and write to file
crimes_counts = pd.DataFrame(
    crimes_merged.groupby(['neighborhood_clean', 'district_clean']).size().rename('counts')
).reset_index()

In [ ]:
# All neighborhood-level Census data sourced from the City of Cincinnati:
# http://www.cincinnati-oh.gov/planning/reports-data/census-demographics/
neighborhood_clean_pops = pd.DataFrame([
    ['Avondale', 12466],
    ['Bond Hill', 6972],
    ['CUF', 16989],
    ['California', 469],
    ['Camp Washington', 1343],
    ['Carthage', 2733],
    ['Clifton', 8304],
    ['College Hill', 14133],
    ['Columbia Tusculum', 1304],
    ['Corryville', 3327],
    ['Downtown', 4850],
    ['East End', 1518],
    ['East Price Hill', 15340],
    ['East Walnut Hills', 3794],
    ['East Westwood', 2445],
    ['English Woods', 405],
    ['Evanston', 9158],
    ['Hartwell', 4640],
    ['Hyde Park', 13356],
    ['Kennedy Heights', 4847],
    ['Linwood', 875],
    ['Lower Price Hill', 1075],
    ['Madisonville', 9141],
    ['Millvale', 2399],
    ['Mt. Adams', 1481],
    ['Mt. Airy', 8779],
    ['Mt. Auburn', 4904],
    ['Mt. Lookout', 4814],
    ['Mt. Washington', 11711],
    # Combining North Avondale and Paddock Hills because of shapefiles
    # ['North Avondale', 3229],
    ['North Avondale - Paddock Hills', 4188],
    ['North Fairmount', 1812],
    ['Northside', 7467],
    ['Oakley', 10429],
    ['Over-the-Rhine', 6064],
    # Combining North Avondale and Paddock Hills because of shapefiles
    # ['Paddock Hills', 959],
    ['Pendleton', 900],
    ['Pleasant Ridge', 8083],
    ['Queensgate', 142],
    ['Riverside', 2340],
    ['Roselawn', 6440],
    ['Sayler Park', 2765],
    ['Sedamsville', 680],
    ['South Cumminsville', 801],
    ['South Fairmount', 2368],
    ['Spring Grove Village', 1964],
    ['Villages at Roll Hill', 1916],
    ['Walnut Hills', 6495],
    ['West End', 6627],
    ['West Price Hill', 17155],
    ['Westwood', 29950],
    ['Winton Hills', 4787]
], columns = ['neighborhood_clean', 'population'])

In [ ]:
# Join crime and population counts
crimes_clean_counts_and_pops = crimes_counts.merge(neighborhood_clean_pops, on='neighborhood_clean')
crimes_clean_counts_and_pops.to_csv('crimes_clean_counts_and_pops.csv', index=False)